In [ ]:
# executar esse script somente no google colab
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

In [ ]:
!pip install chromedriver-autoinstaller

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd
import openpyxl
import requests
from io import BytesIO
import chromedriver_autoinstaller

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

# ------------------ Dependencias --------------------------

# Configurando o WebDriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# definindo o caminho para chromedriver de acordo com sua configuração
chromedriver_autoinstaller.install()

# configura o webdriver
navegador = webdriver.Chrome(options=chrome_options)


# URL da página alvo
url = "https://www.sspds.ce.gov.br/estatisticas-2-3/"

# Lista para armazenar os links dos PDFs
pdf_links = []

# Obtendo o conteúdo da página
navegador.get(url)
soup = BeautifulSoup(navegador.page_source, 'html.parser')

# Encontrando todos os links na página
links = soup.find_all('a', class_='box')

# print(links)


# Filtrando apenas os links que contêm meses e anos
for link in links:
    if any(mes_ano in link['href'] for mes_ano in ["Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho", "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro"]):
        pdf_links.append(link['href'])

# print(pdf_links)

# Iterando sobre os links dos PDFs
# for pdf_link in pdf_links:
    # Fazendo o download do PDF
    # response = requests.get(pdf_link)
    # pdf_data = response.content

# Iterando sobre os links dos PDFs
for pdf_link in pdf_links:
    navegador.get(pdf_link)
    sleep(1)  # Aguarde o carregamento da página
    # Simule um clique no link de download
    download_link = navegador.find_element(By.CSS_SELECTOR, 'a[href$=".pdf"]')
    #download_link.click()
    sleep(5)


navegador.quit()


In [ ]:
!pip install pdfplumber

In [ ]:
import os

caminho_atual = os.getcwd()
print("O diretório atual é:", caminho_atual)

O diretório atual é: /content


In [ ]:
import pdfplumber
import pandas as pd
import os

# Lista para armazenar os dados extraídos dos PDFs
dados_pdf = []

# Diretório onde os PDFs estão salvos
diretorio_pdf = '/content'

# Iterar sobre os PDFs baixados
for pdf_file in os.listdir(diretorio_pdf):
    if pdf_file.endswith(".pdf"):
        pdf_path = os.path.join(diretorio_pdf, pdf_file)
        with pdfplumber.open(pdf_path) as pdf:
            texto_pdf = ""
            for page in pdf.pages:
                texto_pdf += page.extract_text()
        # Aqui você pode processar o texto extraído conforme necessário
        # Por exemplo, dividir o texto em linhas e extrair os dados relevantes
        dados_pdf.append({'Nome do PDF': pdf_file, 'Texto': texto_pdf})

# Criar DataFrame a partir dos dados extraídos
df = pd.DataFrame(dados_pdf)

# Salvar o DataFrame como uma planilha do Excel
excel_path = '/content/resultado.xlsx'
df.to_excel(excel_path, index=False)

print("Planilha do Excel criada com sucesso:", excel_path)


In [ ]:
import os
import pdfplumber
import pandas as pd

# Função para extrair e estruturar os dados do texto do PDF
def extrair_dados_pdf(texto):
    # Divida o texto em linhas
    linhas = texto.split('\n')

    # Lista para armazenar os dados estruturados
    dados = []

    # Iterar sobre as linhas e extrair os dados
    for linha in linhas:
        # Exemplo de processamento básico - ajuste conforme necessário
        if 'ID:' in linha:
            # Extrair os dados da linha
            dados_linha = linha.split(',')
            # Adicionar os dados à lista
            dados.append({
                'ID': dados_linha[0].split(':')[1].strip(),
                'AIS': dados_linha[1].split(':')[1].strip(),
                'MUNICÍPIO': dados_linha[2].split(':')[1].strip(),
                'NATUREZA DO FATO': dados_linha[3].split(':')[1].strip(),
                'MEIO EMPREGADO': dados_linha[4].split(':')[1].strip(),
                'DATA': dados_linha[5].split(':')[1].strip(),
                'SEXO': dados_linha[6].split(':')[1].strip(),
                'IDADE': dados_linha[7].split(':')[1].strip()
            })

    return dados

# Diretório onde os PDFs estão salvos
diretorio_pdf = "/content"

# Lista para armazenar os dados extraídos dos PDFs
dados_pdf = []

# Iterar sobre os PDFs baixados
for pdf_file in os.listdir(diretorio_pdf):
    if pdf_file.endswith(".pdf"):
        pdf_path = os.path.join(diretorio_pdf, pdf_file)
        with pdfplumber.open(pdf_path) as pdf:
            texto_pdf = ""
            for page in pdf.pages:
                texto_pdf += page.extract_text()
        dados_pdf.extend(extrair_dados_pdf(texto_pdf))

# Criar DataFrame a partir dos dados extraídos
df = pd.DataFrame(dados_pdf)

# Caminho para salvar a planilha do Excel
excel_path = '/content/resultado.xlsx'

# Salvar o DataFrame como uma planilha do Excel
df.to_excel(excel_path, index=False)

print("Planilha do Excel criada com sucesso:", excel_path)


In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00


In [ ]:
import PyPDF2
import pandas as pd

# Abrir o arquivo PDF
pdf_file = open('CVLI-Diario-para-o-Site-Janeiro-de-2024.pdf', 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)

# Extrair o texto de todas as páginas
text = ''
for page_num in range(len(pdf_reader.pages)):
    page = pdf_reader.pages[page_num]
    text += page.extract_text()

# Dividir o texto em linhas
lines = text.split('\n')

# Criar uma lista de listas com os dados
data = []
headers = ['ID', 'AIS', 'MUNICÍPIO', 'NATUREZA DO FATO', 'MEIO EMPREGADO', 'DATA', 'SEXO', 'IDADE']

for line in lines:
    if line.startswith('ID AIS'):
        continue  # Ignorar a linha de cabeçalho original
    elif not line.startswith('DADOS CONSOLIDADOS') and not line.startswith('VÍTIMAS DE CRIMES VIOLENTOS LETAIS INTENCIONAIS'):
        line_data = line.split()
        if len(line_data) == len(headers):
            data.append(line_data)

# Criar um DataFrame com os dados
df = pd.DataFrame(data, columns=headers)

# Salvar o DataFrame em um arquivo CSV
df.to_excel('planilha.xlsx', index=False)